In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import time
import cv2

from pySINDy.sindypde import SINDyPDE
from pySINDy import SINDy
from pySINDy.sindybase import SINDyBase
from pySINDy.sindylibr import SINDyLibr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def plotUV(Un, Vn, du_dt, dv_dt,x):

    ##--SAMPLING--##
    s=[78,56]
    U_ds=np.zeros(s)
    V_ds=np.zeros(s)
    d_u=np.zeros(s)
    d_v=np.zeros(s)
    for i in range(1,s[0]):
        for j in range(1,s[1]):
            U_ds[i,j]=Un[10*i,10*j]
            V_ds[i,j]=Vn[10*i,10*j]
            d_u[i,j]=np.reshape(du_dt,file_dim)[10*i,10*j]
            d_v[i,j]=np.reshape(dv_dt,file_dim)[10*i,10*j]

    fig, ax = plt.subplots(figsize=(s[1],s[0]))

    x_pos = np.arange(0,s[1],1)
    y_pos = np.arange(0,s[0],1)


    ax.quiver(x_pos,y_pos, U_ds[:,:], V_ds[:,:], width=0.001)
    ax.set_title('Plotting motion vectors')
    #plt.show()
    U_ds+=10*d_u
    V_ds+=10*d_v
    fig, ax = plt.subplots(figsize=(s[1],s[0]))

    x_pos = np.arange(0,s[1],1)
    y_pos = np.arange(0,s[0],1)
    ax.quiver(x_pos,y_pos, U_ds[:,:], V_ds[:,:], width=0.0005)
    ax.set_title('Plotting motion vectors')
    filename="%s/%i_%i_Im%i.png"%(savepath,pred_date,pred_times[x],x)
    plt.savefig(filename)
    plt.close()

def trainSINDy(U,V,dx,dy,dt):

    
    model = SINDyPDE(name='SINDyPDE model for Reaction-Diffusion Eqn')
    
    start_train=time.time()
    #model.fit(self, data, poly_degree=2, cut_off=1e-3)
    model.fit({'u': U[:,:,-1*train_size:], 'v': V[:,:,-1*train_size:]}, dt, [dx, dy], space_deriv_order=2, poly_degree=2, sample_rate=0.01, cut_off=0.01, deriv_acc=2)

    print("\n--- Train time %s seconds ---\n" %(time.time() - start_train))

    #print("\n--- Active terms ---\n" )
    size=np.shape(model.coefficients)
    cnt=0
    for i in range(size[0]):
        for j in range(size[1]):
            if (model.coefficients[i,j])!=0:
                #print(model.coefficients[i,j],"--",model.descriptions[i])
                cnt+=1
    print("Train Success..")
    print("--- Active terms %s ---\n" %cnt)
    return U, V, model.coefficients

def testSINDy(U,V,dx,dy,dt,coeff,x):

    
    model2 = SINDyLibr(name='Derived module from sindybase.py for libr computation')
    libx=model2.libr({'u': U[:,:,-1*train_size:], 'v': V[:,:,-1*train_size:]}, dt, [dx,dy], space_deriv_order=2, poly_degree=2, sample_rate=0.01, cut_off=0.01, deriv_acc=2)

    duv_dt=np.matmul(libx,coeff)

    du_dt=duv_dt[:,0]
    dv_dt=duv_dt[:,1]

    U_nxt=np.reshape(U,file_dim)+np.reshape(du_dt,file_dim)
    V_nxt=np.reshape(V,file_dim)+np.reshape(dv_dt,file_dim)
    
    Uname="%s/%i_U%i.csv"%(savepath,pred_date,x)
    Vname="%s/%i_V%i.csv"%(savepath,pred_date,x)
    np.savetxt(Uname,np.reshape(U,file_dim),delimiter=',')
    np.savetxt(Vname,np.reshape(V,file_dim),delimiter=',')
    print("Test Success..")
    #plotUV(U_nxt,V_nxt,du_dt,dv_dt,x)

    return U_nxt, V_nxt

def loadUV(loadpath):
    start_load = time.time()
    U_mat=np.zeros(file_dim)
    V_mat=np.zeros(file_dim)
    print("\n--- Loading UV-Data.. ---\n")
    for i in range (len(inp_times)-1):
            #print("%i_%i00_%i_%i00_u.csv"%(pred_date,pred_times[i],pred_date,pred_times[i+1]))
            #print("%i_%i00_%i_%i00_v.csv"%(pred_date,pred_times[i],pred_date,pred_times[i+1]))
            U_mat = np.dstack([U_mat,pd.read_csv("%s/%i_%s_%i_%s_u.csv"%(loadpath,fnames[i]),sep=',',header=None).values])
            V_mat = np.dstack([V_mat,pd.read_csv("%s/%i_%s_%i_%s_v.csv"%(loadpath,pred_date,inp_times[i],pred_date,inp_times[i+1]),sep=',',header=None).values])
            
    print("\n--- Load complete.. Time: %s seconds ---\n" %(time.time() - start_load))
    
    return U_mat[:,:,-1*train_size:], V_mat[:,:,-1*train_size:]

def loadUV_a(loadpath):
    start_load = time.time()
    U_mat=np.zeros(file_dim)
    V_mat=np.zeros(file_dim)
    print("\n--- Loading UV-Data.. ---\n")
    for i in range (len(fnames)-1):
            print(fnames[i])
            U_mat = np.dstack([U_mat,pd.read_csv("%s/%s_u.csv"%(loadpath,fnames[i]),sep=',',header=None).values])
            V_mat = np.dstack([V_mat,pd.read_csv("%s/%s_v.csv"%(loadpath,fnames[i]),sep=',',header=None).values])
    print("\n--- Load complete.. Time: %s seconds ---\n" %(time.time() - start_load))
    
    return U_mat[:,:,-1*train_size:], V_mat[:,:,-1*train_size:]

def gettimes(pred_time):

    times=[['0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200','1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'],
    ['0010','0110','0210','0310','0410','0510','0610','0710','0810','0910','1010','1110','1210','1310','1410','1510','1610','1710','1810','1910','2010','2110','2210','2310'],
    ['0020','0120','0220','0320','0420','0520','0620','0720','0820','0920','1020','1120','1220','1320','1420','1520','1620','1720','1820','1920','2020','2120','2220','2320'],
    ['0030','0130','0230','0330','0430','0530','0630','0730','0830','0930','1030','1130','1230','1330','1430','1530','1630','1730','1830','1930','2030','2130','2230','2330'],
    ['0040','0140','0240','0340','0440','0540','0640','0740','0840','0940','1040','1140','1240','1340','1440','1540','1640','1740','1840','1940','2040','2140','2240','2340'],
    ['0050','0150','0250','0350','0450','0550','0650','0750','0850','0950','1050','1150','1250','1350','1450','1550','1650','1750','1850','1950','2050','2150','2250','2350']]

    times=np.array(times)

    for i in range(24):
        #print(times[5,i-1])
        if pred_time==times[5,i-1]:

            pred_times=np.hstack([times[:,i],times[:,i+1],times[:,i+2]]) #next 18 frames
            #print("PredTimes:",pred_times)
            inp_times=np.hstack([times[:,i-2],times[:,i-1]]) #prev $train_size frames
            #print("InpTimes:",inp_times[-1*(train_size+1):])
            inp_times=inp_times[-1*(train_size+1):]
            #print("Get Time Success..")
    return inp_times, pred_times


In [2]:
fnames = ['201605291725_88_201605291735_90','201605291825_100_201605291835_102',
'201605291735_90_201605291745_92','201605291835_102_201605291845_104',
'201605291745_92_201605291755_94','201605291845_104_201605291855_106',
'201605291755_94_201605291805_96','201605291855_106_201605291905_108',
'201605291805_96_201605291815_98','201605291905_108_201605291915_110',
'201605291815_98_201605291825_100']
fnames.sort()


##Data Params##
pred_date = 20160529 #YYYYMMDD
#pred_time = '1915' #HHMM(HH50) 
file_dim = [900,900]
train_size = 10

start_prog=time.time() #tic
#inp_times, pred_times = gettimes(pred_time)

##I/O Paths##
loadpath = '../../../Datasets/sindy/For29052016_1915'
savepath = "Outputs/%i"%(pred_date)
os.makedirs(savepath, exist_ok=True)

U,V = loadUV_a(loadpath)




--- Loading UV-Data.. ---

201605291725_88_201605291735_90
201605291735_90_201605291745_92
201605291745_92_201605291755_94
201605291755_94_201605291805_96
201605291805_96_201605291815_98
201605291815_98_201605291825_100
201605291825_100_201605291835_102
201605291835_102_201605291845_104
201605291845_104_201605291855_106
201605291855_106_201605291905_108

--- Load complete.. Time: 4.361156940460205 seconds ---



In [3]:
np.shape(U)

(900, 900, 10)

In [4]:
np.shape(V)

(900, 900, 10)

In [5]:
np.isnan(U).any()

False

In [6]:
np.isnan(V).any()

False

In [ ]:
#Spatio-temporal resolution
x_step = 1
y_step = 1
t_step = 10.0

for i in range(18):
    print("\n--- Sequence %i ---\n" %i)
    #if(i==0):
    U, V, coeff = trainSINDy(U,V,x_step,y_step,t_step)
    U_nxt, V_nxt = testSINDy(U[:,:,-1:],V[:,:,-1:],x_step,y_step,t_step, coeff, i)
    U=np.dstack([U,U_nxt])
    V=np.dstack([V,V_nxt])
    
print("\n--- Exec time %s seconds ---\n" %(time.time() - start_prog)) #tac
print("SINDy complete. Files saved to %s" %(savepath))
